In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# reading the CSV file into pandas dataframe
df = pd.read_csv("d:\gli\dse\data\car-mpg.csv")  

pd.isnull(df).count()

In [ ]:
df  = df.drop("car_name" , axis=1)

In [ ]:
df.describe()

In [ ]:
temp  = pd.DataFrame(df.hp.str.isdigit()) 
temp[temp['hp'] == False] 

In [ ]:
# On inspecting records number 32, 126 etc, we find "?" in the columns. Replace them with "nan"
#Replace them with nan and remove the records from the data frame that have "nan"
df = df.replace('?', np.nan)

In [ ]:
# Since the question marks appear only in the "hp" columns, replace them with the median of the same column
df = df.apply(lambda x: x.fillna(x.median()),axis=0)


In [ ]:
# While reading the data into dataframe, Pandas assigned the data type as object to column "hp" because of the question marks
# The column type continues to be same even after replacing the question marks with the median
# Convert the column type to float64


df['hp'] = df['hp'].astype('float64')  # converting the hp column from object / string type to float

In [ ]:
import seaborn as sns

sns.pairplot(data=df)



In [ ]:
# convert origin to numric values (but that will introduce an order in that column)


df['Origin'] = df['origin'].map(lambda x: {1: 'USA', 2: 'Europe', 3: 'Japan'}.get(x))

In [ ]:
# convert the numeric values to one-hot code to remove the order

df = pd.get_dummies(df, columns=['origin'])

In [ ]:
df.head()

In [ ]:
# Import `train_test_split` from `sklearn.model_selection`
from sklearn.model_selection import train_test_split

# Specify the data 
X=df.iloc[:,1:8]

# Specify the target labels and flatten array
y= df.mpg


# Split the data up in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

y_train =  np.array(y_train)
y_test =  np.array(y_test)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

# Deep Neural Network

In [ ]:
# Using Tensorflow Keras instead of the original Keras

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


ncols = X_train.shape[0]

# Initialize the constructor
model = Sequential()

# Add an first hidden layer 
model.add(Dense(20, activation='relu', kernel_initializer='normal'))

# second hidden layer
#model.add(Dense(10, activation='relu', kernel_initializer='normal'))


# Add an output layer with one neuron and no activation specified
model.add(Dense(1))





In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)


model.compile(loss='mean_squared_error',
              optimizer=optimizer , metrics = ['mae' , 'mse'])
                   

epochs = 50


In [ ]:
history = model.fit(X_train, y_train, epochs=epochs, validation_split = 0.2, verbose = False)

hist  = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
#print(hist)




In [ ]:
import matplotlib.pyplot as plt

plt.plot(hist['mse'])
plt.plot(hist['val_mse'])
plt.legend(("train" , "valid") , loc =0)


In [ ]:
plt.plot(hist['mae'])
plt.plot(hist['val_mae'])
plt.legend(("train" , "valid") , loc =0)

In [ ]:

loss = model.evaluate(X_test, y_test, verbose=False)

In [ ]:
y_pred = np.round(model.predict(X_test))

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test, y_pred)